#  Deaths heatmap
---

Inspired by: https://twitter.com/VictimOfMaths/status/1258783876870176770?s=09

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-packages" data-toc-modified-id="Import-packages-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import packages</a></span></li><li><span><a href="#Import-data" data-toc-modified-id="Import-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import data</a></span></li><li><span><a href="#States-heatmap" data-toc-modified-id="States-heatmap-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>States heatmap</a></span></li><li><span><a href="#Cities-heatmap" data-toc-modified-id="Cities-heatmap-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Cities heatmap</a></span></li></ul></div>

## Import packages

In [1]:
import sys
sys.path.append("..")
sys.path.append("../src")

import yaml
config = yaml.load(open('../src/configs/config.yaml', 'r'))

/Users/fernandascovino/Projects/simulacovid/venvanalysis/lib/python3.7/site-packages/ipykernel_launcher.py:6: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


In [2]:
%reload_ext autoreload
%autoreload 2

from paths import RAW_PATH, TREAT_PATH, OUTPUT_PATH, FIGURES_PATH, THEMES_PATH

from copy import deepcopy
import numpy as np
import pandas as pd
pd.options.display.max_columns = 999

import warnings

warnings.filterwarnings('ignore')

# Plotting
import plotly
import plotly.graph_objs as go
import cufflinks as cf
plotly.offline.init_notebook_mode(connected=True)

# Setting cufflinks
import textwrap
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

# Centering and fixing title
def iplottitle(title, width=40):
    return '<br>'.join(textwrap.wrap(title, width))

# Adding custom colorscales (to add one: themes/custom_colorscales.yaml)
import yaml
custom_colorscales = yaml.load(open(THEMES_PATH / 'custom_colorscales.yaml', 'r'))
cf.colors._custom_scales['qual'].update(custom_colorscales)
cf.colors.reset_scales()

# Setting cuffilinks template (use it with .iplot(theme='custom')
cf.themes.THEMES['custom'] = yaml.load(open(THEMES_PATH / 'cufflinks_template.yaml', 'r'))

## Import data

In [17]:
df = pd.read_csv('http://localhost:7000/v1/raw/csv')
df['last_updated'] = pd.to_datetime(df['last_updated'])
df

,country_iso,country_name,state_id,state_name,city_id,city_name,population,health_system_region,last_updated_number_ventilators,author_number_ventilators,number_ventilators,last_updated_number_beds,author_number_beds,number_beds,last_updated_number_icu_beds,author_number_icu_beds,number_icu_beds,city,last_updated,epidemiological_week,estimated_population_2019,is_last,is_repeated,confirmed_cases,last_available_confirmed_per_100k_inhabitants,last_available_date,last_available_death_rate,deaths,order_for_place,place_type,state,daily_cases,new_deaths,infectious_period_cases,state_notification_rate,notification_rate,active_cases,data_last_refreshed
0,BR,Brasil,AC,Acre,1200013,Acrelândia,15256,Acre,2020-02-01 00:00:00,SUS,NaN,2020-02-01 00:00:00,SUS,3.2,2020-02-01 00:00:00,SUS,NaN,Acrelândia,2020-03-29,14.0,15256.0,False,False,2.0,13.10960,2020-03-29,0.0,0.0,1.0,city,AC,2.0,0.0,2.0,NaN,NaN,2.0,2020-05-13 23:49:35
1,BR,Brasil,AC,Acre,1200013,Acrelândia,15256,Acre,2020-02-01 00:00:00,SUS,NaN,2020-02-01 00:00:00,SUS,3.2,2020-02-01 00:00:00,SUS,NaN,Acrelândia,2020-03-30,14.0,15256.0,False,False,6.0,39.32879,2020-03-30,0.0,0.0,2.0,city,AC,4.0,0.0,6.0,NaN,NaN,6.0,2020-05-13 23:49:35
2,BR,Brasil,AC,Acre,1200013,Acrelândia,15256,Acre,2020-02-01 00:00:00,SUS,NaN,2020-02-01 00:00:00,SUS,3.2,2020-02-01 00:00:00,SUS,NaN,Acrelândia,2020-03-31,14.0,15256.0,False,False,7.0,45.88359,2020-03-31,0.0,0.0,3.0,city,AC,1.0,0.0,7.0,NaN,NaN,7.0,2020-05-13 23:49:35
3,BR,Brasil,AC,Acre,1200013,Acrelândia,15256,Acre,2020-02-01 00:00:00,SUS,NaN,2020-02-01 00:00:00,SUS,3.2,2020-02-01 00:00:00,SUS,NaN,Acrelândia,2020-04-01,14.0,15256.0,False,False,8.0,52.43838,2020-04-01,0.0,0.0,4.0,city,AC,1.0,0.0,8.0,NaN,NaN,8.0,2020-05-13 23:49:35
4,BR,Brasil,AC,Acre,1200013,Acrelândia,15256,Acre,2020-02-01 00:00:00,SUS,NaN,2020-02-01 00:00:00,SUS,3.2,2020-02-01 00:00:00,SUS,NaN,Acrelândia,2020-04-02,14.0,15256.0,False,False,8.0,52.43838,2020-04-02,0.0,0.0,5.0,city,AC,0.0,0.0,8.0,NaN,NaN,8.0,2020-05-13 23:49:35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77284,BR,Brasil,TO,Tocantins,1722107,Xambioá,11540,Região de Saúde do Médio Araguaia,2020-02-01 00:00:00,SUS,NaN,2020-02-01 00:00:00,SUS,5.6,2020-02-01 00:00:00,SUS,NaN,Xambioá,2020-05-09,19.0,11540.0,False,False,2.0,17.33102,2020-05-09,0.0,0.0,1.0,city,TO,2.0,0.0,2.0,1.0,1.0,2.0,2020-05-13 23:49:35
77285,BR,Brasil,TO,Tocantins,1722107,Xambioá,11540,Região de Saúde do Médio Araguaia,2020-02-01 00:00:00,SUS,NaN,2020-02-01 00:00:00,SUS,5.6,2020-02-01 00:00:00,SUS,NaN,Xambioá,2020-05-10,20.0,11540.0,False,False,4.0,34.66205,2020-05-10,0.0,0.0,2.0,city,TO,2.0,0.0,4.0,1.0,1.0,4.0,2020-05-13 23:49:35
77286,BR,Brasil,TO,Tocantins,1722107,Xambioá,11540,Região de Saúde do Médio Araguaia,2020-02-01 00:00:00,SUS,NaN,2020-02-01 00:00:00,SUS,5.6,2020-02-01 00:00:00,SUS,NaN,Xambioá,2020-05-11,20.0,11540.0,True,False,5.0,43.32756,2020-05-11,0.0,0.0,3.0,city,TO,1.0,0.0,5.0,1.0,1.0,5.0,2020-05-13 23:49:35
77287,BR,Brasil,TO,Tocantins,1722107,Xambioá,11540,Região de Saúde do Médio Araguaia,2020-02-01 00:00:00,SUS,NaN,2020-02-01 00:00:00,SUS,5.6,2020-02-01 00:00:00,SUS,NaN,Xambioá,2020-05-12,20.0,11540.0,False,True,5.0,43.32756,2020-05-11,0.0,0.0,4.0,city,TO,0.0,0.0,5.0,1.0,1.0,5.0,2020-05-13 23:49:35


In [4]:
# df = pd.read_csv('data/raw/br_cases_city_daily_notification_rate.csv', index_col=0)
# df

## States heatmap

In [18]:
def get_rolling_amount(grp):
    return grp.rolling(5, min_periods=1, on='last_updated')['deaths'].mean()

df_deaths = df[~df['deaths'].isnull()][['state_id', 'last_updated', 'deaths']]\
                                                .groupby(['state_id', 'last_updated'])['deaths']\
                                                .sum()\
                                                .reset_index()

df_deaths['rolling_deaths_new'] = df_deaths.groupby('state_id', 
                                                    as_index=False, 
                                                    group_keys=False)\
                                           .apply(get_rolling_amount)

In [19]:
df_deaths

,state_id,last_updated,deaths,rolling_deaths_new
0,AC,2020-03-17,0.0,0.0
1,AC,2020-03-18,0.0,0.0
2,AC,2020-03-19,0.0,0.0
3,AC,2020-03-20,0.0,0.0
4,AC,2020-03-21,0.0,0.0
...,...,...,...,...
1653,TO,2020-05-09,11.0,9.4
1654,TO,2020-05-10,12.0,10.0
1655,TO,2020-05-11,14.0,11.0
1656,TO,2020-05-12,14.0,12.0


In [28]:
def _df_to_plotly(df):
    return {'z': df.values.tolist(),
            'x': df.columns.tolist(),
            'y': df.index.tolist()}
    
def plot_deaths_heatmap(t, place_type, title):
    
    df_heatmap = t.reset_index()\
                          .pivot(index=place_type, 
                                  columns='last_updated', 
                                  values='rolling_deaths_new')\
                           .apply(lambda x: x/x.max(), axis=1)\
                           .fillna(0)\
                           .dropna(how='all')

    # remove days with all states zero
    df_heatmap = df_heatmap.loc[:, (df_heatmap != 0).any(axis=0)]
    print(len(df_heatmap))

    fig = go.Figure(data=go.Heatmap(_df_to_plotly(df_heatmap), 
                                    colorscale='temps'), 
                    layout={'width': 600,
                            'height': 800,
                            'title': title})
    fig.show()

In [29]:
plot_deaths_heatmap(df_deaths, 'state_id',
                    title='Distribuição de novas mortes nas UFs (mavg = 5 days)')

27


In [30]:
def plot_deaths_heatmap2(t, place_type, title,save_img=False):
    
    df_heatmap = t.reset_index()\
                          .pivot(index=place_type, 
                                  columns='last_updated', 
                                  values='rolling_deaths_new')\
                           .fillna(0)\
                           .apply(lambda x: x/x.max(), axis=1)\
                           .dropna(how='all')

    # remove days with all states zero
    df_heatmap = df_heatmap.loc[:, (df_heatmap != 0).any(axis=0)]
    print(len(df_heatmap))
    states_total_deaths = [t[t['state_id'] ==x]['deaths'].max() for x in df_heatmap.index]
    idy = np.argsort(states_total_deaths) 
    
    trace1 = go.Heatmap(_df_to_plotly(df_heatmap.iloc[idy,:]), 
                                    colorscale='temps')


    trace2 = go.Bar(x=np.array(states_total_deaths)[idy],y=df_heatmap.iloc[idy,:].index,
                    xaxis="x2",yaxis="y2",orientation='h')
    d = [trace1,trace2]
    layout = go.Layout(title=title,
        plot_bgcolor='rgba(0,0,0,0)',
        xaxis=dict(
            domain=[0, 0.8]
        ),
        xaxis2=dict(
            domain=[0.85, 1]
        ),
        yaxis=dict(tickmode='linear'
        ),
        yaxis2=dict(tickmode='linear',anchor='x2'
        )
        )

    fig = go.Figure(data=d, layout=layout)
    if save_img:
        pio.write_image(fig,'Mortes_Brasil.jpeg',format='jpeg',width=1920, height=1080)
    fig.show()

In [31]:
plot_deaths_heatmap2(df_deaths,
                     place_type='state_id',
                    title='Distribuição de novas mortes nas UFs (mavg = 5 days)',
                    save_img=False)

27


## Cities heatmap

In [35]:
! pip install psutil

     |████████████████████████████████| 449 kB 447 kB/s eta 0:00:01
  Created wheel for psutil: filename=psutil-5.7.0-cp37-cp37m-macosx_10_14_x86_64.whl size=226893 sha256=7a669e7c30e5302ed45a1f20c0956a5b9f201affca07dda7459066f63feaf513
  Stored in directory: /Users/fernandascovino/Library/Caches/pip/wheels/b6/e7/50/aee9cc966163d74430f13f208171dee22f11efa4a4a826661c
Successfully built psutil
You should consider upgrading via the '/Users/fernandascovino/Projects/simulacovid/venvanalysis/bin/python3.7 -m pip install --upgrade pip' command.


In [36]:
def get_rolling_amount(grp):
    return grp.rolling(5, min_periods=1, on='last_updated')['deaths'].mean()

df_cities_deaths = df[~df['deaths'].isnull()][['state_id','city_name', 'last_updated', 'deaths']]\
                                                .groupby(['state_id','city_name', 'last_updated'])['deaths']\
                                                .sum()\
                                                .reset_index()

df_cities_deaths['rolling_deaths_new'] = df_cities_deaths.groupby('city_name', 
                                                    as_index=False, 
                                                    group_keys=False)\
                                           .apply(get_rolling_amount)

In [39]:
import plotly.io as pio

def plot_cities_deaths_heatmap(t, state, place_type,min_deaths, title, colors='temps',save_img=False):
    t = t[t['state_id']==state]
    df_heatmap = t.reset_index()\
                          .pivot(index=place_type, 
                                  columns='last_updated', 
                                  values='rolling_deaths_new')\
                           .fillna(0)\
                           .apply(lambda x: x/x.max(), axis=1)\
                           .dropna(how='all')

    # remove days with all states zero
    df_heatmap = df_heatmap.loc[:, (df_heatmap != 0).any(axis=0)]
    print(len(df_heatmap))

    city_total_deaths = [t[t['city_name'] ==x]['deaths'].max() for x in df_heatmap.index]
    idy = np.argsort(city_total_deaths)
    idx = np.array(city_total_deaths,dtype=np.int32)[idy] > min_deaths

    
    try:
        trace1 = go.Heatmap(_df_to_plotly(df_heatmap.iloc[idy,:].loc[idx,:].drop('0',axis=1)), 
                                        colorscale=colors,showscale=False)
    except:
        trace1 = go.Heatmap(_df_to_plotly(df_heatmap.iloc[idy,:].loc[idx,:]), 
                                        colorscale=colors,showscale=False)

    trace2 = go.Bar(x=np.array(city_total_deaths)[idy][idx],y=df_heatmap.iloc[idy,:].loc[idx,:].index,
                    xaxis="x2",yaxis="y2",orientation='h')
    d = [trace1,trace2]
    layout = go.Layout(title=title,
        plot_bgcolor='rgba(0,0,0,0)',
        xaxis=dict(
            domain=[0, 0.7]
        ),
        xaxis2=dict(
            domain=[0.8, 1],
            showticklabels=True
        ),
        yaxis=dict(tickmode='linear'
        ),
        yaxis2=dict(tickmode='linear',anchor='x2'
        )
        
    )

    fig = go.Figure(data=d, layout=layout)
#     if save_img:
#         pio.write_image(fig,'Mortes_{}.jpeg'.format(state),format='jpeg',width=1920, height=1080)
    fig.show()

In [40]:
UF='CE'
plot_cities_deaths_heatmap(df_cities_deaths,
                           state=UF,
                           place_type='city_name',
                           min_deaths=10,
                           title='Distribuição de novas mortes em {}'.format(UF),
                           colors='temps',
                           save_img=True)


91


In [41]:
import geopandas as gpd

In [42]:
import plotly.express as px

fig = px.choropleth( locationmode="country names",  locations = ["Brazil"])
fig.show()